In [12]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch
import torch.linalg as la

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from data.ModelNet40 import ModelNet40

In [22]:
train_dataset = ModelNet40('train')
test_dataset = ModelNet40('test')
cross_dataset = ModelNet40('cross-category')

5114
dict_keys(['pointcloud_a', 'pointcloud_b', 'transformation_matrix', 'class'])


In [23]:
from utils.visualization import visualize_pointclouds

In [24]:
visualize_pointclouds(train_dataset[1000]['pointcloud_a'].T,train_dataset[1000]['pointcloud_b'].T,0.02)

Output()

# DataLoader

In [26]:
from torch.utils.data.dataloader import DataLoader

In [27]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=False)

# Pointmixer

In [8]:
from model.pointmixer import PointMixer

In [9]:
cloud_a, cloud_b = torch.rand(2, 3, 1024), torch.rand(2, 3, 1024)
pm = PointMixer(two_pooling=False)
mixer_a, mixer_b = pm(cloud_a, cloud_b)
print("output shape is", mixer_a.shape, mixer_b.shape)

output shape is torch.Size([2, 128, 1024]) torch.Size([2, 128, 1024])


# Feature Interaction Model

In [19]:
from model.featureinteraction import FeatureInteraction

In [20]:
feat1, feat2 = torch.rand((2, 128, 1024)), torch.rand((2, 128, 1024))
feature_interaction = FeatureInteraction()
fusion1, fusion2 = feature_interaction(feat1, feat2)
print("fusion1 and fusion2 output shapes are", fusion1.shape, fusion2.shape)
assert fusion1.shape == (2, 256, 1024)
assert fusion2.shape == (2, 256, 1024)

fusion1 and fusion2 output shapes are torch.Size([2, 256, 1024]) torch.Size([2, 256, 1024])


# Decoder

In [21]:
from model.decoder import Decoder

In [22]:
x = torch.rand((2, 256, 1024)) # (batches, channels, number of points)
decoder = Decoder(256)
output = decoder(x)
print("output shape is ", output.shape)
assert output.shape == (2, 3, 1024)

output shape is  torch.Size([2, 3, 1024])


# Forward Pass

In [27]:
for i,j in enumerate(train_dataloader):
    cloud_a, cloud_b = j["input"].float(), j["output"].float()
    pm = PointMixer(two_pooling=False)
    mixer_a, mixer_b = pm(cloud_a, cloud_b)
    print("output shape is", mixer_a.shape, mixer_b.shape)
    
    feat1, feat2 = mixer_a, mixer_b
    feature_interaction = FeatureInteraction()
    fusion1, fusion2 = feature_interaction(feat1, feat2)
    print("\nfusion1 and fusion2 output shapes are", fusion1.shape, fusion2.shape)
    
    x = fusion1 # (batches, channels, number of points)
    decoder = Decoder(256)
    output = decoder(x)
    print("\noutput shape cloud A ", output.shape)
    
    x = fusion2 # (batches, channels, number of points)
    decoder = Decoder(256)
    output = decoder(x)
    print("\noutput shape cloud B ", output.shape)
    
    break

output shape is torch.Size([2, 128, 1024]) torch.Size([2, 128, 1024])

fusion1 and fusion2 output shapes are torch.Size([2, 256, 1024]) torch.Size([2, 256, 1024])

output shape cloud A  torch.Size([2, 3, 1024])

output shape cloud B  torch.Size([2, 3, 1024])


# Combined Model

In [28]:
from model.genreg import GenReg

In [30]:
model = GenReg()
for i,j in enumerate(train_dataloader):
    Ag,Bg = model(j["input"], j["output"])
    print("\noutput shape cloud A ", Ag.shape)
    print("\noutput shape cloud B ", Bg.shape)   
    break


output shape cloud A  torch.Size([2, 3, 1024])

output shape cloud B  torch.Size([2, 3, 1024])


# Training

In [110]:
config = {
    "device": "cuda",
    'experiment_name': "test",
    'batch_size': 2,
    'resume_ckpt': None,
    'learning_rate': 0.1,
    'max_epochs': 10,
    'print_every_n': 1000,
    'validate_every_n': 500
}

In [36]:
from train_3depn import main, train

In [37]:
main(config)

Using CPU
its set


In [106]:
# Declare device
device = torch.device('cpu')
if torch.cuda.is_available() and config['device'].startswith('cuda'):
    device = torch.device(config['device'])
    print('Using device:', config['device'])
else:
    print('Using CPU')

# Create Dataloaders
train_dataset = ModelNet40('train')
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
    batch_size=config['batch_size'],   # The size of batches is defined here
    shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
      # This is an implementation detail to speed up data uploading to the GPU
    # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
)

val_dataset = ModelNet40('cross-category')
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,     # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
    batch_size=config['batch_size'],   # The size of batches is defined here
    shuffle=False,   # During validation, shuffling is not necessary anymore
    
    # worker_init_fn=val_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
)

# Instantiate model
model = GenReg()

# Load model if resuming from checkpoint
if config['resume_ckpt'] is not None:
    model.load_state_dict(torch.load(config['resume_ckpt'], map_location='cpu'))

# Move model to specified device
model.to(device)

print("its set")

Using CPU
its set


In [107]:
from pathlib import Path

import numpy as np
import torch

from model.genreg import GenReg
from data.ModelNet40 import ModelNet40
from training.loss import GenRegLoss


def train(model, train_dataloader, val_dataloader, device, config):
    # TODO: Declare loss and move to device; we need both smoothl1 and pure l1 losses here

    loss_criterion = GenRegLoss()

    # TODO: Declare optimizer with learning rate given in config
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

    # Here, we follow the original implementation to also use a learning rate scheduler -- it simply reduces the learning rate to half every 20 epochs
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    # TODO: Set model to train
    model.train()

    best_loss_val = np.inf

    # Keep track of running average of train loss for printing
    train_loss_running = 0.
    

    for epoch in range(config['max_epochs']):
        for batch_idx, batch in enumerate(train_dataloader):
            # TODO: Move batch to device, set optimizer gradients to zero, perform forward pass
            # ShapeNet.move_batch_to_device(batch, device)

            optimizer.zero_grad()

            A = batch['input'].float()
            B = batch['output'].float()

            Ag, Bg = model(A,B)
            

            # TODO: Compute loss, Compute gradients, Update network parameters
            loss = loss_criterion(A, Ag, B, Bg)
            optimizer.step()
            
          

            # Logging
            train_loss_running += loss.item()
            iteration = epoch * len(train_dataloader) + batch_idx

            if iteration % config['print_every_n'] == (config['print_every_n'] - 1):
                print(f'[{epoch:03d}/{batch_idx:05d}] train_loss: {train_loss_running / config["print_every_n"]:.6f}')
                train_loss_running = 0.

            # Validation evaluation and logging
            if iteration % config['validate_every_n'] == (config['validate_every_n'] - 1):
                # TODO: Set model to eval

                model.eval()

                # Evaluation on entire validation set
                loss_val = 0.
                for batch_val in val_dataloader:
                    # ShapeNet.move_batch_to_device(batch_val, device)

                    with torch.no_grad():
                        A = batch_val['input']
                        B = batch_val['output']

                        Ag, Bg = model(A,B)

                        

                    loss_val += loss_criterion(A, Ag, B, Bg).item()

                loss_val /= len(val_dataloader)
                if loss_val < best_loss_val:
                    torch.save(model.state_dict(), f'./runs/{config["experiment_name"]}/model_best.ckpt')
                    best_loss_val = loss_val

                print(f'[{epoch:03d}/{batch_idx:05d}] val_loss: {loss_val:.6f} | best_loss_val: {best_loss_val:.6f}')

                # TODO: Set model back to train
                model.train()

        scheduler.step()

In [108]:
train(model, train_dataloader, val_dataloader, device, config)

predicted
calculating loss
loss calculated
loss backward
finish
set
